In [ ]:
import pickle
import pandas as pd
import numpy as np
import math

## Load Data

In [ ]:
with open('song_user.dict', 'rb') as handle:
    unserialized_song_user = pickle.load(handle) #dict
    
with open('user_song.dict', 'rb') as handle:
    unserialized_user_song = pickle.load(handle) #dict
    
with open('song_person.dict', 'rb') as handle:
    unserialized_song_person = pickle.load(handle) #dict
    
with open('person_song.dict', 'rb') as handle:
    unserialized_person_song = pickle.load(handle) #dict

with open('song_id.txt', 'rb') as handle:
    unserialized_song_id = pickle.load(handle) #numpy.ndarray
    
with open('user_id.txt', 'rb') as handle:
    unserialized_user_id = pickle.load(handle) #numpy.ndarray
    
with open('person_id.txt', 'rb') as handle:
    unserialized_person_id = pickle.load(handle) #numpy.ndarray

## Sort Nodes By Degree

In [ ]:
# key: song, value: number of users listening to it + number of person relating to its creation
song_degree_dict = {}
for (k,v) in unserialized_song_user.items():
    song_degree_dict[k] = v
for (k,v) in unserialized_song_person.items():
    if k in song_degree_dict.keys():
        song_degree_dict[k] = song_degree_dict[k] + v
    else:
        song_degree_dict[k] = v
song_degree = [(k,len(v)) for (k,v) in song_degree_dict.items()]

# sort by degree in descending order
song_degree.sort(key = lambda x : -x[1])

In [ ]:
# key: person, value: number of songs they create
person_degree = [(k,len(v)) for (k,v) in unserialized_person_song.items()]

# sort by degree in descending order
person_degree.sort(key = lambda x : -x[1])

In [ ]:
# key: user, value: number of songs they listen to
user_degree = [(k,len(v)) for (k,v) in unserialized_user_song.items()]

# sort by degree in descending order
user_degree.sort(key = lambda x : -x[1])

## Construct Subnetworks

In [ ]:
# construct sparse subnetwork
# find the nodes
sparse_net_song_nodes_holder = song_degree[-int(len(song_degree)*0.2):] #song_id is the first item in the tuple element of the returned list
sparse_net_song_nodes = [node_holder[0] for node_holder in sparse_net_song_nodes_holder]

sparse_net_user_nodes_holder = user_degree[-int(len(user_degree)*0.2):]
sparse_net_user_nodes = [node_holder[0] for node_holder in sparse_net_user_nodes_holder]

sparse_net_person_nodes_holder = person_degree[-int(len(person_degree)*0.2):]
sparse_net_person_nodes = [node_holder[0] for node_holder in sparse_net_person_nodes_holder]

sparse_net_nodes = sparse_net_song_nodes + sparse_net_user_nodes + sparse_net_person_nodes

# build the subnetwork --> find the edges
sparse_net_edge = [] # a list of pairs (node1, node2)
connect = [] # a list of nodes that the node1 should connect to
for i in sparse_net_nodes: # (node1, node2) and (node2, node1) both exist
    if i in unserialized_song_user:
        connect = unserialized_song_user[i]
        if i in unserialized_song_person:
            connect.extend(unserialized_song_person[i])
    elif i in unserialized_song_person:
        connect = unserialized_song_person[i]
        if unserialized_song_user:
            connect.extend(unserialized_song_user[i])
    elif i in unserialized_user_song:
        connect = unserialized_user_song[i]
    elif i in unserialized_person_song:
        connect = unserialized_person_song[i]
    else:
        print('Error: key error!')
    new_edges = [(i, j) for j in connect]
    sparse_net_edge.extend(new_edges)

# compute the density heuristic 
sparse_net_density = len(sparse_net_edge)/(len(sparse_net_song_nodes)*len(sparse_net_user_nodes))

In [ ]:
sparse_net_density

In [ ]:
# construct dense subnetwork
# find the nodes
dense_net_song_nodes_holder = song_degree[:int(len(song_degree)*0.2)] #song_id is the first item in the tuple element of the returned list
dense_net_song_nodes = [node_holder[0] for node_holder in dense_net_song_nodes_holder]

dense_net_user_nodes_holder = user_degree[:int(len(user_degree)*0.2)]
dense_net_user_nodes = [node_holder[0] for node_holder in dense_net_user_nodes_holder]

dense_net_person_nodes_holder = person_degree[:int(len(person_degree)*0.2)]
dense_net_person_nodes = [node_holder[0] for node_holder in dense_net_person_nodes_holder]

dense_net_nodes = dense_net_song_nodes + dense_net_user_nodes + dense_net_person_nodes

# build the subnetwork --> find the edges
dense_net_edge = [] # a list of pairs (node1, node2)
connect = [] # a list of nodes that the node1 should connect to
for i in dense_net_nodes: # (node1, node2) and (node2, node1) both exist
    if i in unserialized_song_user:
        connect = unserialized_song_user[i]
        if i in unserialized_song_person:
            connect.extend(unserialized_song_person[i])
    elif i in unserialized_song_person:
        connect = unserialized_song_person[i]
        if unserialized_song_user:
            connect.extend(unserialized_song_user[i])
    elif i in unserialized_user_song:
        connect = unserialized_user_song[i]
    elif i in unserialized_person_song:
        connect = unserialized_person_song[i]
    else:
        print('Error: key error!')
    new_edges = [(i, j) for j in connect]
    dense_net_edge.extend(new_edges)

# compute the density heuristic 
dense_net_density = len(dense_net_edge)/(len(dense_net_song_nodes)*len(dense_net_user_nodes))

In [ ]:
dense_net_density